In [33]:
import os
import time 
from openai import OpenAI
import json
from dotenv import load_dotenv

load_dotenv()

MY_ENV_VAR = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=MY_ENV_VAR)
print("client ready")

client ready


In [39]:
System_Prompt_question_gen = "Below is the user's goal or task, based on clear understanding give the following in JSON:\n- List of top(max 5) very important questions to ask the user with options to improve the goal statement and make it less vague. Questions to be asked to remove vagueness. Its type can be \"choice\",  \"input\" or \"date\", if input then give options as empty list(but prefer choice over input). Do not ask questions only if it's very well described(respond with empty list for questions). \n- State weather if it is a product, service or invalid goal (in goal_type). Format - {\"questions\":[{\"question\":\"\",\"type\":\"\",\"options\":[\"\",\"\"],},{}],\"goal_type\":\"\"}"

question_gen_few_shot = [ {
                    "role": "user",
                    "content": "Location: Oakland, CA;\nGoal: I want a new car",
                },
                {
                    "role": "system",
                    "content": "{\"questions\":[{\"question\":\"What type of car do you want?\",\"type\":\"choice\",\"options\":[\"SUV\", \"Sedan\", \"Truck\", \"Coupe\"]},{\"question\":\"What is the budget for the car?\",\"type\":\"choice\",\"options\":[\"$1000\", \"$2000\", \"$3000\", \"$4000\"]},{\"question\":\"Which brand of car do you prefer?\",\"type\":\"choice\",\"options\":[\"Toyota\", \"Honda\", \"Ford\", \"Chevrolet\"]}],\"goal_type\":\"product\"}",
                },
                {
                    "role": "user",
                    "content": "Goal:I want to do an internship in UC Davis in the fiels of Microbiology.",
                },
                { "role": "system",
                 "content": "{\"questions\":[{\"question\":\"What is the duration of the internship?\",\"type\":\"choice\",\"options\":[\"1 month\", \"2 months\", \"3 months\", \"4 months\"]},{\"question\":\"What type of internship do you want?\",\"type\":\"input\",\"options\":[]},{\"question\":\"What is the field of study?\",\"type\":\"input\",\"options\":[]}],\"goal_type\":\"service\"}"
                }
                ]

In [40]:
"{\"questions\":[{\"question\":\"What type of car do you want?\",\"type\":\"choice\",\"options\":[\"SUV\", \"Sedan\", \"Truck\", \"Coupe\"]},{\"question\":\"What is the budget for the car?\",\"type\":\"choice\",\"options\":[\"$1000\", \"$2000\", \"$3000\", \"$4000\"]},{\"question\":\"Which brand of car do you prefer?\",\"type\":\"choice\",\"options\":[\"Toyota\", \"Honda\", \"Ford\", \"Chevrolet\"]}],\"goal_type\":\"product\"}"

'{"questions":[{"question":"What type of car do you want?","type":"choice","options":["SUV", "Sedan", "Truck", "Coupe"]},{"question":"What is the budget for the car?","type":"choice","options":["$1000", "$2000", "$3000", "$4000"]},{"question":"Which brand of car do you prefer?","type":"choice","options":["Toyota", "Honda", "Ford", "Chevrolet"]}],"goal_type":"product"}'

In [42]:
GOAL = "I want to shift my items from Kochi to mumbai"

start_time = time.time()

response = client.chat.completions.create(
  model= "gpt-3.5-turbo",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": System_Prompt_question_gen},
    *question_gen_few_shot,
    {"role": "user", "content": f"Goal:{GOAL}"},
  ]
)

end_time = time.time()
print(response)

print(f'Total Tokens:{response.usage.total_tokens}, Prompt Token {response.usage.prompt_tokens}, Completion Token {response.usage.completion_tokens}')
print(f'Time Taken: {end_time - start_time} Sec\n')

response = json.loads(response.choices[0].message.content)
print(f"Goal: {GOAL}")
print(json.dumps(response, indent=4))

ChatCompletion(id='chatcmpl-8xwitdvOlTvFZKKakmNQVJijlTUXW', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"questions":[{"question":"When do you plan to shift your items from Kochi to Mumbai?","type":"date","options":[]},{"question":"What type of items are you looking to shift?","type":"input","options":[]},{"question":"Do you have any specific preferences for the moving company?","type":"choice","options":["Yes", "No"]}],"goal_type":"service"}', role='assistant', function_call=None, tool_calls=None))], created=1709297775, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_2b778c6b35', usage=CompletionUsage(completion_tokens=80, prompt_tokens=400, total_tokens=480))
Total Tokens:480, Prompt Token 400, Completion Token 80
Time Taken: 3.105532169342041 Sec

Goal: I want to shift my items from Kochi to mumbai
{
    "questions": [
        {
            "question": "When do you plan to shift your items from Koc

In [30]:
question_merge_sys_prompt = "Given the user's goal and the questions asked to the user with its answers, merge the questions into the goal to make it less vague. If the goal is already well described, respond with the goal as it is. Respond in JSON, Format - {\"merged_goal\":\"\"}"

question_merge_few_shot = [ {
                    "role": "user",
                    "content": "Goal: I want a new car,  Answers: What type of car do you want? - SUV, Which brand of car do you prefer? - Toyota, Location: Oakland, CA",
                },
                {
                    "role": "system",
                    "content": "{\"merged_goal\":\"I want a new Toyota SUV car in Oakland, CA\"}",
                }]

In [32]:
GOAL = "I want to do an internship in UC Davis this summer."



start_time = time.time()

response = client.chat.completions.create(
  model= "gpt-3.5-turbo",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": question_merge_sys_prompt},
    *question_merge_few_shot,
    {"role": "user", "content": f"Goal:{GOAL},Answers: What type of internship do you want? - Research, What is the duration of the internship? - 3 months, What is the field of study? - Microbiology"},
  ]
)

end_time = time.time()
print(response)

print(f'Total Tokens:{response.usage.total_tokens}, Prompt Token {response.usage.prompt_tokens}, Completion Token {response.usage.completion_tokens}')
print(f'Time Taken: {end_time - start_time} Sec\n')

response = json.loads(response.choices[0].message.content)
print(f"Goal: {GOAL}")
print(json.dumps(response, indent=4))

ChatCompletion(id='chatcmpl-8xozVf2BxXGxoXI35gpJLyizl94rE', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"merged_goal":"I want to do a 3-month research internship in Microbiology at UC Davis this summer."}', role='assistant', function_call=None, tool_calls=None))], created=1709268053, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_86156a94a0', usage=CompletionUsage(completion_tokens=24, prompt_tokens=177, total_tokens=201))
Total Tokens:201, Prompt Token 177, Completion Token 24
Time Taken: 1.338939905166626 Sec

Goal: I want to do an internship in UC Davis this summer.
{
    "merged_goal": "I want to do a 3-month research internship in Microbiology at UC Davis this summer."
}
